### import python pibraries

In [1]:
pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
#for data frames and opertaion
import pandas as pd
import numpy as np

#for visualisation
%matplotlib inline
import matplotlib.pyplot as plt

### data loading

In [5]:
#loading of the datasets
df = pd.read_csv('clinicData.csv')
# drop unnamed column from the clinical dataset.
df.dropna(how='all', axis='columns', inplace=True)

### data cleaning

#### strip non-nmeric characters

In [6]:
df.replace('<', '', regex=True, inplace=True)
df.replace('mm', '', regex=True, inplace=True)
df.replace('>', '', regex=True, inplace=True)
df.replace('~','.', regex=True, inplace=True)
df.replace('-','.', regex=True, inplace=True)
df.replace('-','.', regex=True, inplace=True)
df.replace('-','.', regex=True, inplace=True)

#### check columns with missing values

In [7]:
df.isna().sum()

group           0
name            0
generative      0
age             0
PD depth        0
mSBI score      0
SBI score      10
MBL count       0
PLI score       0
BOP probing     0
CAL score       0
FMBS score      1
dtype: int64

Column SBI score has the highest values of null values.
there are a total 10 missing values.

##### handling missing values

In [8]:
df['SBI score'].fillna(0,inplace=True)
df['FMBS score'].fillna(0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   group        60 non-null     object 
 1   name         60 non-null     object 
 2   generative   60 non-null     object 
 3   age          60 non-null     int64  
 4   PD depth     60 non-null     object 
 5   mSBI score   60 non-null     int64  
 6   SBI score    60 non-null     float64
 7   MBL count    60 non-null     object 
 8   PLI score    60 non-null     int64  
 9   BOP probing  60 non-null     object 
 10  CAL score    60 non-null     object 
 11  FMBS score   60 non-null     object 
dtypes: float64(1), int64(3), object(8)
memory usage: 5.8+ KB


####  data manipulation

###### target column
     0 --> negative(healthy)
     1 --> positive (not-healthy)

In [9]:
df.loc[df['BOP probing'] == '.', 'BOP probing'] = 0
df.loc[df['BOP probing'] == '+', 'BOP probing'] = 1

As part of the data cleaning process, we have to convert text data to numerical because computations are best suited for numerical inputs.

In [10]:
cols_to_convert = ['PD depth','MBL count','CAL score','FMBS score',]
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce', axis=1)
df[cols_to_convert].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PD depth    60 non-null     float64
 1   MBL count   60 non-null     float64
 2   CAL score   60 non-null     float64
 3   FMBS score  60 non-null     float64
dtypes: float64(4)
memory usage: 2.0 KB


### descriptive statistics and data distribution

In [11]:
# Return list of numerical columns that are not necessarily continuous to find any outliers.
num_cols = [col for col in df.columns if (df[col].dtype=='int64' or df[col].dtype=='float64') and col != 'BOP probing']
df[num_cols].describe()

,age,PD depth,mSBI score,SBI score,MBL count,PLI score,CAL score,FMBS score
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,36.366667,1.966667,0.883333,0.866667,1.708333,0.683333,2.100000,1.100000
std,11.244534,1.261243,1.075011,1.383519,1.667769,0.833446,1.755379,0.439568
min,21.000000,1.000000,0.000000,0.000000,0.500000,0.000000,1.000000,0.000000
25%,29.000000,1.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.000000
50%,35.000000,1.000000,0.500000,0.000000,0.600000,0.000000,1.000000,1.000000
75%,39.250000,2.875000,1.250000,2.000000,2.650000,1.000000,2.500000,1.000000
max,73.000000,4.500000,3.000000,4.000000,5.000000,2.000000,6.000000,2.000000


##### visualisation

In [18]:
features = ['generative','age', 'PD depth', 'mSBI score',
            'SBI score', 'MBL count', 'PLI score', 'CAL score','FMBS score']

In [24]:
https://intellipaat.com/blog/tutorial/machine-learning-tutorial/svm-algorithm-in-python/

SyntaxError: invalid syntax (56713651.py, line 1)

# Linear Regression algorothim 1

# Naive Bayes algorithim 2

# Logistic Regression algorithim 3

# regression algorithim 4
